# RecursiveCharacterTextSplitter 사용

In [15]:
# 형법 pdf
file_path = './dataset/criminal-law.pdf'

# Langchain의 PyPDFLoader를 이용
from langchain_community.document_loaders import PyPDFLoader
import re
from langchain_core.documents import Document
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# PDF 로더 객체 생성
loader = PyPDFLoader(file_path)

# pdf의 각 페이지를 저장
pages = []

# 비동기 방식으로 pdf 페이지 로드
async for page in loader.alazy_load():
    pages.append(page)

In [16]:
# 맨앞 두 페이지는 쓸모없어서 버림
pages = pages[2:]

# 페이지 내용을 하나의 문자열로
full_text = "\n".join(page.page_content for page in pages)

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document 
import re

# Patterns for structure identification
edition_pattern = re.compile(r'(제\d+편 [^\n]+)')
chapter_pattern = re.compile(r'(제\d+장 [^\n]+)')
# Pattern to capture article number and title, e.g., "제21조(정당방위)"
article_pattern = re.compile(r'(제\d+조(?:의\d+)?\s*\(.+?\))')

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # 청크 크기 (조정 가능)
    chunk_overlap=150,  # 청크 간 중첩 크기 (조정 가능)
    length_function=len,
    # 분할 기준 우선순위: 조 -> 장 -> 편 -> 문단 -> 줄바꿈 -> 공백
    separators=[
        r"(제\d+조(?:의\d+)?\s*\(.+?\))", # 조 (Article)
        r"(제\d+장 [^\n]+)",             # 장 (Chapter)
        r"(제\d+편 [^\n]+)",             # 편 (Edition)
        "\n\n",                         # 문단 구분
        "\n",                           # 줄바꿈
        " ",                            # 공백
    ],
    is_separator_regex=True,
)

# Split the entire text content directly
text_chunks = text_splitter.split_text(full_text)

# Process chunks to create Document objects and assign metadata
chunks = []
current_edition = ""
current_chapter = ""
current_article = "" # 현재 조(Article) 추적 추가

for text_chunk in text_chunks:
    metadata = {}
    chunk_content_to_check = text_chunk # 메타데이터 검색을 위해 원본 청크 사용

    # 청크 시작 부분에서 편/장/조 정보 업데이트 시도
    edition_match = edition_pattern.match(chunk_content_to_check)
    if edition_match:
        current_edition = edition_match.group(1).strip()
        # 편 정보가 업데이트되면 하위 장/조 정보 초기화 (선택적)
        current_chapter = ""
        current_article = ""

    chapter_match = chapter_pattern.match(chunk_content_to_check)
    if chapter_match:
        current_chapter = chapter_match.group(1).strip()
        # 장 정보가 업데이트되면 하위 조 정보 초기화 (선택적)
        current_article = ""

    article_match = article_pattern.match(chunk_content_to_check)
    if article_match:
        current_article = article_match.group(1).strip()

    # 현재까지 파악된 최신 편/장/조 정보를 메타데이터에 할당
    if current_edition:
        metadata["edition"] = current_edition
    if current_chapter:
        metadata["chapter"] = current_chapter
    # 조(Article) 정보는 청크 시작 부분이 아니더라도 포함될 수 있으므로,
    # match 대신 search를 사용하고, current_article도 업데이트
    article_search_match = article_pattern.search(chunk_content_to_check)
    if article_search_match:
         # 검색된 첫번째 조 정보를 메타데이터에 저장하고, 현재 조 정보로 업데이트
        matched_article = article_search_match.group(1).strip()
        metadata["article"] = matched_article
        current_article = matched_article # 다음 청크를 위해 업데이트
    elif current_article: # 청크 내에 명시적 조항이 없으면 이전 조항 유지
         metadata["article"] = current_article


    # 페이지 정보는 이 방식으로는 정확히 알 수 없으므로 제거하거나 다른 방법 고려
    # metadata["page"] = ?

    chunks.append(Document(page_content=text_chunk, metadata=metadata))

In [18]:
print(f"Total chunks created: {len(chunks)}")
if chunks:
    print("\n--- First Chunk ---")
    print(f"Metadata: {chunks[0].metadata}")
    print(f"Content:\n{chunks[0].page_content[:500]}...") # Print first 500 chars

if len(chunks) > 1:
    print("\n--- Second Chunk ---")
    print(f"Metadata: {chunks[1].metadata}")
    print(f"Content:\n{chunks[1].page_content[:500]}...") # Print first 500 chars

Total chunks created: 80

--- First Chunk ---
Metadata: {'article': '제1조(범죄의 성립과 처벌)'}
Content:
법제처                                                            3                                                       국가법령정보센터
「형법」
 
                    제1편 총칙
                       제1장 형법의 적용범위
 
제1조(범죄의 성립과 처벌)제1조(범죄의 성립과 처벌) ①범죄의 성립과 처벌은 행위 시의 법률에 의한다.
②범죄 후 법률의 변경에 의하여 그 행위가 범죄를 구성하지 아니하거나 형이 구법보다 경한
때에는 신법에 의한다.
③재판확정 후 법률의 변경에 의하여 그 행위가 범죄를 구성하지 아니하는 때에는 형의 집행
을 면제한다.
 
제2조(국내범)제2조(국내범) 본법은 대한민국영역 내에서 죄를 범한 내국인과 외국인에게 적용한다.
 
제3조(내국인의 국외범)제3조(내국인의 국외범) 본법은 대한민국영역 외에서 죄를 범한 내국인에게 적용한다.
 ...

--- Second Chunk ---
Metadata: {'article': '제6조(대한민국과 대한민국 국민에 대한 국외범)'}
Content:
제6조(대한민국과 대한민국 국민에 대한 국외범) 본법은 대한민국영역 외에서 대한민국 또는 대
한민국 국민에 대하여 전조에 기재한 이외의 죄를 범한 외국인에게 적용한다. 단 행위지의 법
률에 의하여 범죄를 구성하지 아니하거나 소추 또는 형의 집행을 면제할 경우에는 예외로 한
다.
 
형법
[시행 2010.10.16] [법률 제10259호, 2010.4.15, 일부개정]
법제처                                                            4                                              

In [19]:
load_dotenv()
api_key = os.environ.get("OPENAI_API_KEY")

In [20]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [21]:
vectorstore = Chroma.from_documents(
  documents = chunks,
  embedding = embeddings,
  persist_directory = "./chroma_db" # 저장할 디렉토리
)

print(f"successfully embedded {vectorstore._collection.count()} documents into ChromaDB")

successfully embedded 207 documents into ChromaDB


In [22]:
# 벡터 저장소 로드 테스트
db = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)
print(f"Loaded ChromaDB with {db._collection.count()} document")

Loaded ChromaDB with 207 document


In [23]:
# 유사도 검색 테스트
query = "정당방위의 요건은 무엇인가?"
docs = vectorstore.similarity_search(query, k=3) # 상위 3개

print(f"\n--- similarity search result for '{query}' ---")
for doc in docs:
  print(f"Metadata: {doc.metadata}")
  print(f"Content Preview: {doc.page_content[:200]}...") # 내용 미리보기
  print("-"*20)


--- similarity search result for '정당방위의 요건은 무엇인가?' ---
Metadata: {'article': '제225조(공문서등의 위조ㆍ변조)'}
Content Preview: 제225조(공문서등의 위조ㆍ변조)제225조(공문서등의 위조ㆍ변조) 행사할 목적으로 공무원 또는 공무소의 문서 또는 도화를 위
조 또는 변조한 자는 10년 이하의 징역에 처한다. <개정 1995.12.29>
 
제226조(자격모용에 의한 공문서 등의 작성)제226조(자격모용에 의한 공문서 등의 작성) 행사할 목적으로 공무원 또는 공무소의 자격을 모
용하여 문...
--------------------
Metadata: {'article': '제225조(공문서등의 위조ㆍ변조)'}
Content Preview: 제225조(공문서등의 위조ㆍ변조)제225조(공문서등의 위조ㆍ변조) 행사할 목적으로 공무원 또는 공무소의 문서 또는 도화를 위
조 또는 변조한 자는 10년 이하의 징역에 처한다. <개정 1995.12.29>
 
제226조(자격모용에 의한 공문서 등의 작성)제226조(자격모용에 의한 공문서 등의 작성) 행사할 목적으로 공무원 또는 공무소의 자격을 모
용하여 문...
--------------------
Metadata: {'article': '제12조(강요된 행위)'}
Content Preview: 농아자의 행위는 형을 감경한다.
 
제12조(강요된 행위)제12조(강요된 행위) 저항할 수 없는 폭력이나 자기 또는 친족의 생명 신체에 대한 위해를 방어
할 방법이 없는 협박에 의하여 강요된 행위는 벌하지 아니한다.
 
제13조(범의)제13조(범의) 죄의 성립요소인 사실을 인식하지 못한 행위는 벌하지 아니한다. 단, 법률에 특별
한 규정이 있는 경우에는 예...
--------------------
